In [1]:
import os
import logging
import src.config as config
from src.logging_setup import configure_logging
from src.orchestrator import (
    process_patients_segmentation, 
    process_patients_registration,
    orchestrate_pet_processing,
    calculate_suv,
)
from src.demographics import create_patient_info_dataframe
from src.suv_analysis import compute_mean_suv_across_patients

logger=configure_logging()

In [2]:
#First step - segmentation
ct_masks_dict = process_patients_segmentation(
    data_folder=config.DATA_FOLDER,
    logger=logger,
    limit=2,
    save_temp=False,
    load_temp=True,
    temp_file="ct_masks_dict.pkl"
)

logger.info("First step - Segmentation - complete")

In [3]:
#Second step - registration
pet_ct_dict = process_patients_registration(
    data_folder=config.DATA_FOLDER,
    logger=logger,
    target_description_pet=config.TARGET_DESCRIPTION_PET,
    target_description_number_ct=config.TARGET_DESCRIPTION_NUMBER_CT,
    target_study=config.TARGET_STUDY,
    limit=6,
    load_dict=True,
    dict_file="pet_ct_dict.pkl"
)

logger.info("Second step - Registration - complete")

In [4]:
#Third step: computing suv
masked_pet_images_dict, masked_images_dict = orchestrate_pet_processing(
    ct_dict=ct_masks_dict,
    pet_dict=pet_ct_dict,
    padding=config.PADDING,
    ct_masks_flipped=config.CT_MASKS_FLIPPED
)

suv_volume_dict = calculate_suv(
    config.DATA_FOLDER,
    masked_pet_images_dict=None,
    logger=logger,
    load_temp=True
)


logger.info("Third step - Computing suv - complete")


#Computing suv by spinal level
mean_suv_by_vertebra_by_patient, mean_suv_by_vertebral_level_across_patients = compute_mean_suv_across_patients(suv_volume_dict, masked_images_dict)
    

logger.info("Pipeline complete")
